In [1]:
# notebook to calculate and save TEM diagnostics
# This assumes CESM(CAM) data have already been organized into zonal mean fluxes
# Uzm, THzm, VTHzm, Vzm, UVzm, UWzm, Wzm 
# note that here we are calculating the E-P fluxes on model interface levels, which is ok
# in the stratosphere but not in the troposphere.  If interested in tropospheric
# E-P flux diagnostics, make sure they have been interpolated to pressure already.

# initial notebook coding by Dan Marsh 16 Dec 2022

In [2]:
import xarray as xr
import numpy as np
from datetime import date
import matplotlib.pyplot as plt

In [3]:
from tem4cam import calc_tem

In [4]:
# open input file
# note: for processing multiple files, simpy use xr.open_mfdataset()
test_file = '/glade/scratch/fvitt/FWsc2000climo_f09_TEM_test05/run/FWsc2000climo_f09_TEM_test05.cam.h6.0001-01-01-00000.nc'
ds = xr.open_dataset(test_file)

In [5]:
#iterate over the times in a dataset

for count, value in enumerate(ds.time.values):
    if count == 0:
        print('first date', value)
        dstem0 = calc_tem(ds.squeeze().isel(time=count))
    else:
        print(count, value)
        dstem = calc_tem(ds.squeeze().isel(time=count))
        dstem0 = xr.concat([dstem0, dstem],'time')

dstem0.attrs = ds.attrs

dstem0.attrs['created'] = str(date.today())


first date 0001-01-01 00:00:00
1 0001-01-02 00:00:00
2 0001-01-03 00:00:00
3 0001-01-04 00:00:00
4 0001-01-05 00:00:00
5 0001-01-06 00:00:00
6 0001-01-07 00:00:00
7 0001-01-08 00:00:00
8 0001-01-09 00:00:00
9 0001-01-10 00:00:00
10 0001-01-11 00:00:00
11 0001-01-12 00:00:00
12 0001-01-13 00:00:00
13 0001-01-14 00:00:00
14 0001-01-15 00:00:00
15 0001-01-16 00:00:00
16 0001-01-17 00:00:00
17 0001-01-18 00:00:00
18 0001-01-19 00:00:00
19 0001-01-20 00:00:00
20 0001-01-21 00:00:00
21 0001-01-22 00:00:00
22 0001-01-23 00:00:00
23 0001-01-24 00:00:00
24 0001-01-25 00:00:00
25 0001-01-26 00:00:00
26 0001-01-27 00:00:00
27 0001-01-28 00:00:00
28 0001-01-29 00:00:00
29 0001-01-30 00:00:00
30 0001-01-31 00:00:00
31 0001-02-01 00:00:00
32 0001-02-02 00:00:00
33 0001-02-03 00:00:00
34 0001-02-04 00:00:00
35 0001-02-05 00:00:00
36 0001-02-06 00:00:00
37 0001-02-07 00:00:00
38 0001-02-08 00:00:00
39 0001-02-09 00:00:00
40 0001-02-10 00:00:00
41 0001-02-11 00:00:00
42 0001-02-12 00:00:00
43 0001-02-1

In [6]:
dstem0

<xarray.Dataset>
Dimensions:    (time: 60, nbnd: 2, lat: 192, ilev: 71)
Coordinates:
    zlon       float64 0.0
  * time       (time) object 0001-01-01 00:00:00 ... 0001-03-01 00:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * ilev       (ilev) float64 4.5e-06 7.42e-06 1.223e-05 ... 967.5 985.1 1e+03
Dimensions without coordinates: nbnd
Data variables: (12/13)
    date       (time) int32 10101 10102 10103 10104 ... 10226 10227 10228 10301
    datesec    (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    time_bnds  (time, nbnd) object 0001-01-01 00:00:00 ... 0001-03-01 00:00:00
    uzm        (time, ilev, lat) float32 0.0 -1.31 -2.564 ... -0.4593 0.0
    vzm        (time, ilev, lat) float32 0.0 0.366 0.2003 ... -0.3601 -0.205 0.0
    epfy       (time, ilev, lat) float64 1.896e-32 0.003906 ... 2.747e-25
    ...         ...
    vtem       (time, ilev, lat) float32 0.0 0.2177 -0.07739 ... 0.05807 0.0
    wtem       (time, ilev, lat) float64 4.805e+12 0.03637 ... -4.991e+11
    psitem     (time, ilev, lat) float64 0.0 13.62 32.09 ... 2.65e+08 0.0
    utendepfd  (time, ilev, lat) float64 5.778e+24 0.0003285 ... 9.2e+23
    utendvtem  (time, ilev, lat) float64 -0.0 -3.166e-05 ... 8.461e-06 0.0
    utendwtem  (time, ilev, lat) float64 0.0 2.248e-06 ... 1.6e-05 -0.0
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              FWsc2000climo_f09_TEM_test05
    logname:           fvitt
    host:              cheyenne4
    initial_file:      /glade/p/cesmdata/cseg/inputdata/cesm2_init/f.e21.FWsc...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     not_set
    time_period_freq:  day_1
    created:           2022-12-16

In [9]:
# write output to a netcdf file
dstem0.to_netcdf('FWsc2000climo_f09_TEM_test05.diag.nc', unlimited_dims='time')

In [10]:
!ncdump -vdate FWsc2000climo_f09_TEM_test05.diag.nc

netcdf FWsc2000climo_f09_TEM_test05.diag {
dimensions:
	time = UNLIMITED ; // (60 currently)
	nbnd = 2 ;
	lat = 192 ;
	ilev = 71 ;
variables:
	double zlon ;
		zlon:_FillValue = NaN ;
		zlon:long_name = "longitude" ;
		zlon:units = "degrees_east" ;
		zlon:bounds = "zlon_bnds" ;
	int64 time(time) ;
		time:long_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 0001-01-01 00:00:00.000000" ;
		time:calendar = "noleap" ;
	int date(time) ;
		date:long_name = "current date (YYYYMMDD)" ;
		date:coordinates = "zlon" ;
	int datesec(time) ;
		datesec:long_name = "current seconds of current date" ;
		datesec:coordinates = "zlon" ;
	double time_bnds(time, nbnd) ;
		time_bnds:_FillValue = NaN ;
		time_bnds:long_name = "time interval endpoints" ;
		time_bnds:coordinates = "zlon" ;
		time_bnds:units = "days since 0001-01-01" ;
	double lat(lat) ;
		lat:_FillValue = NaN ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
	double ilev(ilev) ;
		ilev:_FillValue = NaN ;
	